In [78]:
%load_ext autoreload
%autoreload 2
from gensim.models.doc2vec import Doc2Vec
import sys
sys.path.append('../src')
from ParsePDF import PDF_loader
import numpy as np
from sklearn.preprocessing import normalize
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
model = Doc2Vec(corpus_file='../data/courpus_file', epochs = 40, vector_size=100, window=6, min_count=1, workers=7)

In [5]:
model.infer_vector(['machine'])

array([ 1.7908009e-02, -9.1574453e-03,  3.3688541e-03,  2.0823214e-02,
        2.1315016e-02,  3.5233263e-02, -1.8597908e-02,  4.4522077e-02,
       -1.4698349e-02,  1.5550865e-03,  3.6303487e-03,  2.9490810e-02,
        1.3537992e-02,  2.7086314e-02,  1.3951334e-02, -5.5082027e-02,
        7.3846132e-02,  5.1045537e-02,  1.2348005e-02, -1.0294285e-01,
        8.6747315e-03,  3.0619793e-02,  1.2751141e-02, -7.1544448e-05,
        2.0126428e-02,  1.3862474e-02, -1.0850973e-01, -8.9515842e-02,
        2.2238682e-03, -4.9372643e-02,  5.5510327e-02, -1.4346634e-02,
       -6.4082681e-03, -2.4354503e-02, -2.1232991e-02,  1.3731923e-02,
       -8.6705424e-03, -4.9231425e-03, -3.5955925e-02,  2.6690625e-02,
        3.5056577e-03,  2.9078345e-03,  1.3990891e-02, -3.6592696e-02,
        2.6648136e-02, -3.9192140e-02,  1.1137772e-02, -6.5525420e-02,
        2.5302961e-02, -1.3404121e-02,  6.4710394e-02, -6.5002181e-02,
       -3.2218870e-02, -2.2520958e-02, -9.5500210e-03,  6.0840193e-02,
      

In [126]:
model.dv[0]

array([-1.1109936 , -0.19470303,  2.313719  ,  0.27533114,  5.973465  ,
        4.042419  , -1.8177285 ,  3.7120667 , -2.6151226 ,  2.0617886 ,
        0.8988389 ,  3.1373858 , -1.7692358 , -4.1760116 , -4.4828367 ,
        3.7058697 ,  6.493455  ,  2.1543314 , -8.154132  ,  2.5318298 ,
       -4.637727  ,  0.48426503,  0.72579783,  3.604631  , -4.7688518 ,
        1.519642  , -2.6531272 , -5.04848   ,  4.8203588 ,  1.7745881 ,
       -0.48171294, -0.21351641,  0.37965572, -1.5484892 ,  2.1202745 ,
        0.43647212, -2.9572475 , -2.2481163 , -4.3069196 ,  0.6392312 ,
       -1.4532793 , -0.21880873, -2.3211224 , -1.1015761 ,  5.515307  ,
       -4.5001955 ,  2.9437807 , -2.1172984 ,  2.3468223 ,  1.766686  ,
       -3.976104  ,  2.9239128 , -1.8492473 , -1.4064186 ,  7.156241  ,
        5.075026  , -1.8470958 ,  2.951773  , -1.1225611 ,  5.533775  ,
        4.777734  ,  4.0212307 , -3.0997796 , -4.2468348 , -5.507576  ,
       -3.094892  , -0.20007586, -3.3519328 , -1.0301901 , -1.78

In [118]:
def vectorize_string(text: str):
    words = text.split(' ')
    word_vecs = [model.wv[word] for word in words if word in model.wv.key_to_index.keys()]
    search_vector = np.mean(word_vecs, axis=0)
    return search_vector

def get_similarity(vector):
    norm_vec = normalize(vector.reshape(1,-1))
    norm_dv = normalize(model.dv.vectors)
    similar = np.inner(norm_vec, norm_dv)
    return similar

def search(text:str, top_n:int=5):
    search_vector = vectorize_string(text)
    sims = get_similarity(search_vector)[0]
    ids = np.flip(np.argsort(sims))
    results = pd.DataFrame(files.loc[ids[:top_n], 'file_name'].copy())
    results['similarity'] = sims[ids[: top_n]]
    return results

In [123]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

In [134]:
search('language modeling', top_n=10)

,file_name,similarity
393,2209.07935v1.Structure_Preserving_Transformations_for_Practical_Model_based_Systems_Engineering.pdf,0.303074
203,2209.07511v1.Test_Time_Prompt_Tuning_for_Zero_Shot_Generalization_in_Vision_Language_Models.pdf,0.300174
916,2209.09098v1.Deep_tensor_networks_with_matrix_product_operators.pdf,0.291646
1513,2209.10491v1.Unifying_Classification_Schemes_for_Software_Engineering_Meta_Research.pdf,0.289233
1355,2209.10147v1.The_ReturnZero_System_for_VoxCeleb_Speaker_Recognition_Challenge_2022.pdf,0.285484
1139,2209.09645v1.Multi_level_Explanation_of_Deep_Reinforcement_Learning_based_Scheduling.pdf,0.279685
397,2209.07946v1.Transport_in_reservoir_computing.pdf,0.275101
646,2209.08511v1.A_Type_Directed_Dictionary_Passing_Translation_of_Featherweight_Generic_Go.pdf,0.268320
1134,2209.09632v1.A_Reference_Model_for_Common_Understanding_of_Capabilities_and_Skills_in_Manufacturing.pdf,0.266831
212,2209.07526v1.OmniVL_One_Foundation_Model_for_Image_Language_and_Video_Language_Tasks.pdf,0.265699


In [ ]:
files = pd.read_csv('../data/file_order.csv')

In [86]:
files.drop(columns='Unnamed: 0', inplace=True)

In [88]:
files.columns

Index(['file_name'], dtype='object')

In [125]:
model.save('../models/archive_model')

In [133]:
files.loc[files.loc[:, "file_name"].str.contains('2209.')]

,file_name
0,2209.07026v2.Can_We_Solve_3D_Vision_Tasks_Starting_from_A_2D_Vision_Transformer.pdf
1,2209.07027v1.Generalized_Representations_Learning_for_Time_Series_Classification.pdf
2,2209.07028v1.Estimating_large_causal_polytree_skeletons_from_small_samples.pdf
3,2209.07030v1.Model_Guided_Multi_Contrast_Deep_Unfolding_Network_for_MRI_Super_resolution_Reconstruction.pdf
4,2209.07031v1.A_semantic_hierarchical_graph_neural_network_for_text_classification.pdf
5,2209.07034v1.A_Temporal_Densely_Connected_Recurrent_Network_for_Event_based_Human_Pose_Estimation.pdf
6,2209.07035v1.Online_Combinatorial_Auctions_for_Resource_Allocation_with_Supply_Costs_and_Capacity_Limits.pdf
7,2209.07036v1.Langevin_Autoencoders_for_Learning_Deep_Latent_Variable_Models.pdf
8,2209.07037v1.On_error_based_step_size_control_for_discontinuous_Galerkin_methods_for_compressible_fluid_dynamics.pdf
9,2209.07039v1.Sparsity_Inducing_Representations_for_Policy_Decompositions.pdf
